# TO-DO
1. ~~Add section number (alongside letter)~~
1. Add requirements tables
1. ~~Adapt for different moorings~~
1. Auto generate .pdf
1. Add wire rope summary

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
mooring_num = {'W310':1,'N280':2,'S245':3,'L245':4} # set mooring numbers (used for line sections)

# load moorings
mooring_df = {} # allocate mooring dict for dataframes
file_name = 'Mooring_final_v11.xlsx' # get mooring design Excel file
for mooring in mooring_num: # loop over all moorings
    mooring_df[mooring] = pd.read_excel(file_name,sheet_name=mooring,converters={'SERIAL NUMBER':str}) # load in mooring

# load database
dfarcels = pd.read_excel(file_name,sheet_name='arcels')
dfanchors = pd.read_excel(file_name,sheet_name='anchors')
dfrigging = pd.read_excel(file_name,sheet_name='rigging')
dfchains = pd.read_excel(file_name,sheet_name='chains')
dfcms = pd.read_excel(file_name,sheet_name='cms')
dffloats = pd.read_excel(file_name,sheet_name='floats')
dfmiscs = pd.read_excel(file_name,sheet_name='miscs')
dfwires = pd.read_excel(file_name,sheet_name='wires')

df_data = {'arcels':dfarcels,
           'anchors':dfanchors,
           'rigging':dfrigging,
           'chains':dfchains,
           'cms':dfcms,
           'floats':dffloats,
           'miscs':dfmiscs,
           'wires':dfwires}


In [3]:
# function to make tables gives header and data
def make_table(tab_header,tab_data,tab_cap=None,bool_small=False):
    N_row = len(tab_data[0])
    
    str_tab = []
    
    str_tab.append(r'\begin{table}[]')
    if bool_small:
       str_tab.append(r'\scriptsize')
    if tab_cap is not None:
        str_tab.append(r'\caption{'+tab_cap+r'}')
        str_tab.append(r'\label{'+tab_cap+r'}')
    str_tab.append(r'\centering')
    str_tab.append(r'\begin{adjustbox}{max width=\textwidth,max height=0.8\textheight}')
    str_tab.append(r'\centering')
    str_tab.append(r'\begin{tabular}{'+'l'*(1+len(tab_header))+r'}')
    str_tab.append(r'\hline')
    str_tab.append(r' & '.join(tab_header)+ r' \\ \hline')
    for i in range(N_row):
        # str_tab.append(r' & '.join([j[i] for j in tab_data]) + r' \\')
        str_tab.append(r' & '.join([r'{}'.format(j[i]) for j in tab_data]) + r' \\')
    str_tab.append(r'\hline')
    str_tab.append(r'\end{tabular}')
    str_tab.append(r'\end{adjustbox}')
    str_tab.append(r'\end{table}')

    return '\n'.join(str_tab)

In [12]:
# element summary


def tab_element_summary(mooring_df,df_data,mooring):
    df = mooring_df[mooring]

    # drop bottom
    idx = df['TYPE'] != 'bottom'
    df = df.loc[idx] 

    # in-line
    idx = np.logical_not(df['CLAMP-ON'].to_numpy())
    df = df.loc[idx]

    # allocate
    l_name = []
    l_num  = []
    l_len  = []
    l_bouy = []
    l_m    = []
    l_SWL  = []
    l_com  = []

    types = df['TYPE'].to_numpy()
    types_unq = np.unique(types)
    for Type in types_unq:
        itype = types == Type

        df_type = df.loc[itype]
        names = df_type['NAME'].to_numpy()
        names_unq = np.unique(names)
        for name in names_unq:
            iname = names == name
            df_name = df_type.loc[iname]
            
            # get properties
            idata = df_data[Type]['ELEMENT NAME'] == name
            data = df_data[Type].iloc[np.where(idata)[0][0]].to_dict()

            # store
            l_name.append(name.replace(' (UWA)',''))
            l_num.append(f'{np.sum(iname):.0f}')
            
            L = np.sum(df_name['LINE LENGTH'])
            if L==0:
                l_len.append('')
            else:
                l_len.append(f'{L:.1f}')

            B = data['BUOYANCY [kg]']
            if type(B).__name__ == 'str':
                l_bouy.append(B)
            else:
                l_bouy.append(f'{B:.1f}')
            l_m.append('')
            l_SWL.append('')
            l_com.append('')

    tab_header = ['Element','Qty','Length[m]','Buoyancy[kg]','Weight[kg]','SWL[T]','Comments']
    tab_cap = f'{mooring} : Element summary'
    tab_data = [l_name,l_num,l_len,l_bouy,l_m,l_SWL,l_com]
    tab = make_table(tab_header,tab_data,tab_cap)
    return tab



tab_element_summary(mooring_df,df_data,'W310')


'\\begin{table}[]\n\\caption{W310 : Element summary}\n\\label{W310 : Element summary}\n\\centering\n\\begin{adjustbox}{max width=\\textwidth,max height=0.8\\textheight}\n\\centering\n\\begin{tabular}{llllllll}\n\\hline\nElement & Qty & Length[m] & Buoyancy[kg] & Weight[kg] & SWL[T] & Comments \\\\ \\hline\n3M-WagonW & 1 &  & -900.0 &  &  &  \\\\\nRibuck Dual Kit for Sonardyne ORT & 1 &  & -35.0 &  &  &  \\\\\n3/8 chain SL & 1 & 2.0 & -2.3 &  &  &  \\\\\n100 cm Signature Frame & 1 &  & -7.5 &  &  &  \\\\\n75kHz+40in+Ti300kHz & 1 &  & 85.8 &  &  &  \\\\\n14in Panther Plast & 1 &  & 17.6 &  &  &  \\\\\n30in & 2 &  & 149.0 &  &  &  \\\\\nSercel+frame & 1 &  & -10.0 &  &  &  \\\\\nSwivel & 1 &  & -4.0 &  &  &  \\\\\n13mmBS-L-16mmDS & 2 &  & -0.8 &  &  &  \\\\\n150mm Load Ring & 1 &  & -0.8 &  &  &  \\\\\n16mmBS & 5 &  & -0.7 &  &  &  \\\\\n16mmDS & 1 &  & -0.7 &  &  &  \\\\\n16mmDS-L-13mmBS & 1 &  & -0.8 &  &  &  \\\\\n16mmDS-L-16mmDS & 3 &  & -0.8 &  &  &  \\\\\n3/8 wire rope & 4 & 285.5 &

In [4]:
def tab_clampon_sum(mooring_df,df_data,mooring_num,mooring):
    df = mooring_df[mooring]
    num = mooring_num[mooring]

    # clamp-on summary
    df = mooring_df[mooring]

    # drop bottom
    idx = df['TYPE'] != 'bottom'
    df = df.loc[idx] 

    # clamp-on
    idx = df['CLAMP-ON'].to_numpy()
    df = df.loc[idx]

    # allocate
    l_name = []
    l_serial = []
    l_height = []
    l_clamped_to = []
    l_section = []
    l_HAE = []

    N = len(df)
    for i in range(N):
        data = df.iloc[i].to_dict()

        # store
        l_name.append(data['NAME'].replace(' (UWA)','').replace(' (passive)',''))
        serial = data['SERIAL NUMBER']
        if type(serial).__name__ == 'float':
            serial = ''
        l_serial.append(serial)
        height = data['HEIGHT']
        l_height.append(f'{height:.1f}')
        l_clamped_to.append(data['CLAMPED TO'].replace(' (UWA)',''))
        section = data['#']
        if type(section).__name__ == 'float':
            section = ''
        else:
            section = str(num)+section
        l_section.append(section)
        HAE = data['CO HAE']
        l_HAE.append(f'{HAE:.1f}')

    tab_header = ['Name','Serial','H[m]','Clamped-to','Section','Clamp section H[m]']
    tab_data = [l_name,l_serial,l_height,l_clamped_to,l_section,l_HAE]
    tab_cap = f'{mooring} : Clamp-on summary'

    tab = make_table(tab_header,tab_data,tab_cap)
    return tab

tab_clampon_sum(mooring_df,df_data,mooring_num,'W310')

'\\begin{table}[]\n\\caption{W310 : Clamp-on summary}\n\\label{W310 : Clamp-on summary}\n\\centering\n\\begin{adjustbox}{max width=\\textwidth,max height=0.8\\textheight}\n\\centering\n\\begin{tabular}{lllllll}\n\\hline\nName & Serial & H[m] & Clamped-to & Section & Clamp section H[m] \\\\ \\hline\nRBRquartz TP & 213895 & 0.3 & 3M-WagonW &  & 0.3 \\\\\nSonardyne ORT & 262762-001 [65] & 3.3 & Ribuck Dual Kit for Sonardyne ORT &  & 0.3 \\\\\nSonardyne ORT & 262762-002 [64] & 3.3 & Ribuck Dual Kit for Sonardyne ORT &  & 0.3 \\\\\nFLNTUSB & 2997 & 4.5 & 3/8 wire rope & 1A & 0.2 \\\\\nSBE39 T & 3975 & 4.7 & 3/8 wire rope & 1A & 0.4 \\\\\nZn Anode &  & 7.4 & 3/8 wire rope & 1A & 3.1 \\\\\nFLNTUSB & 1835 & 9.3 & 3/8 wire rope & 1A & 5.0 \\\\\nSBE56 T & 7343 & 9.8 & 3/8 wire rope & 1A & 5.5 \\\\\nSBE39 ext. T & 3799 & 14.8 & 3/8 wire rope & 1A & 10.5 \\\\\nFLNTUSB & 2535 & 19.3 & 3/8 wire rope & 1A & 15.0 \\\\\nSBE56 T & 7344 & 19.8 & 3/8 wire rope & 1A & 15.5 \\\\\nSBE39 sync. T & 4456 & 24.8

In [5]:
# Assembly summary
def tab_assembly_sum(mooring_df,df_data,mooring_num,mooring,bool_small=False):
    df = mooring_df[mooring]
    num = mooring_num[mooring]

    # inverse prop
    df = df.iloc[::-1]

    # drop bottom
    idx = df['TYPE'] != 'bottom'
    df = df.loc[idx] 

    # allocate
    l_name = []
    l_len = []
    l_section = []
    l_ser = []
    l_height = []

    N = len(df)
    for i in range(N):
        prop = df.iloc[i].to_dict()
        
        name = prop['NAME']
        Type = prop['TYPE']
        idata = df_data[Type]['ELEMENT NAME'] == name
        data = df_data[Type].iloc[np.where(idata)[0][0]].to_dict()

        # store
        l_name.append(prop['NAME'].replace(' (UWA)','').replace(' (passive)',''))

        L = np.sum(prop['LINE LENGTH'])
        if L==0 or np.isnan(L):
            l_len.append('')
        else:
            l_len.append(f'{L:.1f}')
        
        if prop['CLAMP-ON']:
            HAE = prop['CO HAE']
            HAE = ' ['+f'{HAE:.1f}'+r' m AE]'
        else:
            HAE = ''

        ser = prop['SERIAL NUMBER']
        if type(ser).__name__ == 'float':
            ser = ''    
        l_ser.append(ser) 

        height = prop['HEIGHT']
        l_height.append(f'{height:.1f}' + HAE)

        if prop['LINE']:
            B_fac = prop['LINE LENGTH']
        else:
            B_fac = 1
        
        # B = B_fac*data['BUOYANCY [kg]']
        # if type(B).__name__ == 'str':
        #     B_str = ''
        # else: 
        #     B_str = f'{B:.1f}'
        # l_boy.append(B_str)
    
        section = prop['#']
        if type(section).__name__ == 'float':
            section = ''
        else:
            section = str(num)+section
        l_section.append(section)

    tab_header = ['Element','Serial','Length [m]','Section','Height [in ASB]']
    tab_data = [l_name,l_ser,l_len,l_section,l_height]
    tab_cap = f'{mooring} : Assembly summary'

    tab = make_table(tab_header,tab_data,tab_cap,bool_small)
    return tab

tab_assembly_sum(mooring_df,df_data,mooring_num,'W310')

'\\begin{table}[]\n\\caption{W310 : Assembly summary}\n\\label{W310 : Assembly summary}\n\\centering\n\\begin{adjustbox}{max width=\\textwidth,max height=0.8\\textheight}\n\\centering\n\\begin{tabular}{llllll}\n\\hline\nElement & Serial & Length [m] & Section & Height [in ASB] \\\\ \\hline\n14in Panther Plast &  &  &  & 297.1 \\\\\n16mmDS &  &  &  & 297.0 \\\\\n6mm AmSteel &  & 1.0 & 1E & 296.0 \\\\\nSercel & 126989 &  &  & 295.9 [0.4 m AE] \\\\\nSercel+frame &  &  &  & 295.5 \\\\\n30in & 7-13_Buoy62_300m &  &  & 294.7 \\\\\n16mmDS-L-16mmDS &  &  &  & 294.5 \\\\\nSBE39 ext. TP & 6523 &  & 1D & 293.8 [3.8 m AE] \\\\\nZn Anode &  &  & 1D & 291.9 [1.9 m AE] \\\\\n3/8 wire rope &  & 4.5 & 1D & 290.0 \\\\\n13mmBS-L-16mmDS &  &  &  & 289.8 \\\\\nSignature 1000 & 100244 &  &  & 289.3 [0.5 m AE] \\\\\n100 cm Signature Frame &  &  &  & 288.8 \\\\\n16mmDS-L-13mmBS &  &  &  & 288.6 \\\\\nSBE37 SMP plastic CTP & 9276 &  & 1C & 282.2 [81.6 m AE] \\\\\nSBE56 T & 7402 &  & 1C & 269.9 [69.3 m AE] \\\\

# Merged

In [6]:

# combine dataframes
df_list = [] # append
for mooring in mooring_df:
    df_i = mooring_df[mooring]
    df_i['MOORING'] = mooring

    df_list.append(df_i)

df_merge = pd.concat(df_list).reset_index()


In [7]:
# total instruments
def tab_total_instrument(df_merge):

    # merge and count
    df_group = df_merge.groupby(['NAME', 'MOORING']).size().reset_index(name='count')
    df_pivot = pd.pivot_table(df_group, values='count', index='NAME', columns='MOORING', aggfunc=np.sum, fill_value=0)

    # clean table
    # drop bottom
    idx = np.logical_not(df_pivot.index == 'bottom')
    df_pivot = df_pivot.loc[idx] 

    # sort index
    df_pivot.sort_index()

    # drop '(UWA)'
    df_pivot.index = [i.replace(' (UWA)','').replace(' (passive)','') for i in df_pivot.index]

    # reorder column
    df_pivot = df_pivot[['L245','S245','N280','W310']]

    # add total
    df_pivot['Total'] = df_pivot.sum(axis=1)

    # make table
    tab_header = ['Element']+list(df_pivot.columns)
    tab_data = [list(df_pivot.index)] + [[str(j) for j in df_pivot[i].values] for i in df_pivot]

    tab = make_table(tab_header,tab_data,tab_cap='All : Total count of each element')
    return tab

tab_total_instrument(df_merge)

'\\begin{table}[]\n\\caption{All : Total count of each element}\n\\label{All : Total count of each element}\n\\centering\n\\begin{adjustbox}{max width=\\textwidth,max height=0.8\\textheight}\n\\centering\n\\begin{tabular}{lllllll}\n\\hline\nElement & L245 & S245 & N280 & W310 & Total \\\\ \\hline\n100 cm Signature Frame & 0 & 1 & 0 & 1 & 2 \\\\\n13mmBS & 0 & 3 & 0 & 0 & 3 \\\\\n13mmBS-L-13mmBS & 0 & 1 & 0 & 0 & 1 \\\\\n13mmBS-L-16mmBS & 0 & 3 & 0 & 0 & 3 \\\\\n13mmBS-L-16mmDS & 0 & 0 & 1 & 2 & 3 \\\\\n14in Panther Plast & 2 & 2 & 1 & 1 & 6 \\\\\n150kHz+30in_sph & 0 & 1 & 0 & 0 & 1 \\\\\n150mm Load Ring & 0 & 1 & 1 & 1 & 3 \\\\\n16mmBS & 0 & 2 & 3 & 5 & 10 \\\\\n16mmBS-L-13mmBS & 0 & 4 & 0 & 0 & 4 \\\\\n16mmBS-L-16mmBS & 0 & 1 & 1 & 0 & 2 \\\\\n16mmDS & 0 & 0 & 1 & 1 & 2 \\\\\n16mmDS-L-13mmBS & 0 & 0 & 0 & 1 & 1 \\\\\n16mmDS-L-16mmDS & 1 & 0 & 3 & 3 & 7 \\\\\n3/8 chain SL & 0 & 1 & 1 & 1 & 3 \\\\\n3/8 wire rope & 1 & 4 & 2 & 4 & 11 \\\\\n30in & 0 & 1 & 2 & 2 & 5 \\\\\n35in & 0 & 1 & 0 &

In [8]:
# summary of instruments

def tab_summary_instrument(df_merge):


    for j in range(4):
        df_group = df_merge

        # keep
        if j == 0:
            idx = (df_group['TYPE'] == 'arcels') | (df_group['TYPE'] == 'cms')
            df_group = df_group.loc[idx] 
        else: 
            idx = (df_group['TYPE'] == 'miscs')
            df_group = df_group.loc[idx] 
        
            idx = df_group.NAME.str.contains('SBE')
            if j == 1:
                df_group = df_group.loc[np.logical_not(idx)] 
            else:
                df_group = df_group.loc[idx]    

                idx = df_group.NAME.str.contains('SBE56')
                if j == 2:
                    df_group = df_group.loc[np.logical_not(idx)] 
                else:
                    df_group = df_group.loc[idx]                    

        # drop bottom
        idx = np.logical_not((df_group['NAME'] == 'bottom') | (df_group['NAME'] =='Zn Anode (UWA)'))
        df_group = df_group.loc[idx] 



        # set name as index
        df_group = df_group.set_index('NAME').sort_index()

        # sort serial numbers per group
        df_group = df_group.groupby('NAME', group_keys=False).apply(lambda x: x.sort_values('SERIAL NUMBER'))
        df_group

        # Count the frequency of each letter
        lst = list(df_group.index)
        freq = {x: lst.count(x) for x in set(lst)}

        name_lst = []
        sorted_keys = list(sorted(freq.keys()))

        for freq_i in sorted_keys:
            n = freq[freq_i]
            name_lst += [freq_i + f' [n = {n}]'] + (n-1)*['']

        df_group.index = name_lst

        # keep only serial and mooring
        df_group = df_group[['SERIAL NUMBER','MOORING']]

        # drop '(UWA)' and '(passive)'
        df_group.index = [i.replace(' (UWA)','').replace(' (passive)','') for i in df_group.index]

        # fix NaN serial 
        idx = [type(i).__name__ == 'float' for i in df_group['SERIAL NUMBER']]
        df_group['SERIAL NUMBER'].loc[idx] = ''



        # make table
        tab_header = ['Element','Serial number','Mooring']
        tab_data = [list(df_group.index)] + [[str(j) for j in df_group[i].values] for i in df_group]
        if j == 0:
            tab = make_table(tab_header,tab_data,tab_cap='All: summary of instruments (part 1/4)')
        if j == 1:
            tab += '\n' + make_table(tab_header,tab_data,tab_cap='All: summary of instruments (part 2/4)')
        if j == 2:
            tab += '\n' + make_table(tab_header,tab_data,tab_cap='All: summary of instruments (part 3/4)')
        if j == 3:
            tab += '\n' + make_table(tab_header,tab_data,tab_cap='All: summary of instruments (part 4/4)')
    return tab

tab_summary_instrument(df_merge)

'\\begin{table}[]\n\\caption{All: summary of instruments (part 1/4)}\n\\label{All: summary of instruments (part 1/4)}\n\\centering\n\\begin{adjustbox}{max width=\\textwidth,max height=0.8\\textheight}\n\\centering\n\\begin{tabular}{llll}\n\\hline\nElement & Serial number & Mooring \\\\ \\hline\n100 cm Signature Frame [n = 2] &  & W310 \\\\\n &  & S245 \\\\\n150kHz+30in_sph [n = 1] &  & S245 \\\\\n75kHz+30in_sph [n = 1] &  & N280 \\\\\n75kHz+40in+Ti300kHz [n = 1] &  & W310 \\\\\nADV+MP+uSquid+2B [n = 1] &  & S245 \\\\\nEdgetech MF SD [n = 2] & 49502 & L245 \\\\\n & 53086 & L245 \\\\\nLongranger [n = 2] & 16870 & W310 \\\\\n & 24613 & N280 \\\\\nQuartermaster [n = 1] & 11795 & S245 \\\\\nRibuck Dual Kit for Sonardyne ORT [n = 3] &  & S245 \\\\\n &  & W310 \\\\\n &  & N280 \\\\\nSentinel [n = 1] & 20089 & W310 \\\\\nSignature 1000 [n = 3] & 100244 & W310 \\\\\n & 100460 & L245 \\\\\n & 100608 & S245 \\\\\nSonardyne ORT [n = 7] & **284838-003 [A3]** & N280 \\\\\n & 262762-001 [65] & W310 \

In [9]:
# Simple section summary

def tab_total_section_sum(df_merge,mooring_num):
    df_lines = df_merge

    idx = df_lines['LINE'] == True

    df_lines = df_lines.loc[idx]
    lst = []
    for i in range(len(df_lines)):
        section = df_lines.iloc[i]['#']
        if section == ' ':
            lst.append('')
            
        else:
            number = mooring_num[df_lines.iloc[i]['MOORING']]
            lst.append(str(number)+section)
            


    df_lines = df_lines.reset_index()
    df_lines['#'] = lst

    df_lines = df_lines[['MOORING','#','NAME','LENGTH']]


    # make table
    tab_header = ['Mooring','Section name','Material','Length']
    tab_data = [[str(j) for j in df_lines[i].values] for i in df_lines]
    tab = make_table(tab_header,tab_data,tab_cap='All: simple section summary')
    return tab

print(tab_total_section_sum(df_merge,mooring_num))


\begin{table}[]
\caption{All: simple section summary}
\label{All: simple section summary}
\centering
\begin{adjustbox}{max width=\textwidth,max height=0.8\textheight}
\centering
\begin{tabular}{lllll}
\hline
Mooring & Section name & Material & Length \\ \hline
W310 &  & 3/8 chain SL & 2.0 \\
W310 & 1A & 3/8 wire rope & 26.0 \\
W310 & 1B & 3/8 wire rope & 167.0 \\
W310 & 1C & 3/8 wire rope & 88.0 \\
W310 & 1D & 3/8 wire rope & 4.5 \\
W310 & 1E & 6mm AmSteel & 1.0 \\
N280 &  & 3/8 chain SL & 1.8 \\
N280 & 2A & 3/8 wire rope & 195.0 \\
N280 & 2B & 3/8 wire rope & 62.5 \\
N280 & 2C & 6mm AmSteel & 1.0 \\
S245 &  & 3/8 chain SL & 1.8 \\
S245 & 3A & 3/8 wire rope & 145.0 \\
S245 & 3B & 3/8 wire rope & 32.5 \\
S245 & 3C & 3/8 wire rope & 32.5 \\
S245 & 3D & 3/8 wire rope & 4.5 \\
S245 & 3E & 6mm AmSteel & 4.5 \\
L245 & 4A & 3/8 wire rope & 70.6 \\
\hline
\end{tabular}
\end{adjustbox}
\end{table}


In [10]:
def tab_per_sec(df_merge,mooring_num):
    tab = ''

    # summary per section
    df_lines = df_merge

    idx = [type(i).__name__ != 'float' for  i in df_lines['#']]
    df_lines = df_lines.loc[idx]

    idx = df_lines['#'] != ' '
    df_lines = df_lines.loc[idx]

    lst = []
    for i in range(len(df_lines)):
        section = df_lines.iloc[i]['#']
        number = mooring_num[df_lines.iloc[i]['MOORING']]
        lst.append(str(number)+section)
            


    df_lines = df_lines.reset_index()
    df_lines['#'] = lst

    section_unq = np.unique(df_lines['#'])

    for section in section_unq:
        idx = df_lines['#'] == section
        df_sec = df_lines.loc[idx]
        info_dict = df_sec.loc[df_sec['LINE']].iloc[0]
        cap = 'Summary of '+info_dict['MOORING']+', '+info_dict['#']+', length: '+str(info_dict['LINE LENGTH'])+', material: '+info_dict['NAME']

        df_ele = df_sec.loc[np.logical_not(df_sec['LINE'])]

        if len(df_ele) > 0:
            HAE = df_ele['HEIGHT']-df_ele['SECTION START']-df_ele['LENGTH']/2
            HAE_str = [f'{i:.1f}' for i in HAE]

            idx = [(type(i).__name__ == 'float') for i in df_ele['SERIAL NUMBER']]
            if np.sum(idx) > 0:
                df_ele['SERIAL NUMBER'].loc[idx] = ''

            # remove '(UWA)' and '(passive)'

            df_ele['NAME'] = [i.replace(' (UWA)','').replace(' (passive)','') for i in df_ele['NAME']]
            
            tab_header = ['Name','Serial number','Height [m]','Along Element [m]']
            tab_data = [list(df_ele['NAME']),list(df_ele['SERIAL NUMBER']),[f'{i:.1f}' for i in df_ele['HEIGHT']],HAE_str]
            tab_cap = cap
            
            tab += '\n' + make_table(tab_header,tab_data,tab_cap=tab_cap)

    return tab

In [13]:
content = r'''\documentclass{article}
\usepackage{graphicx} % Required for inserting images
\usepackage{tabularx}
\usepackage{adjustbox}
\usepackage[margin=.5in]{geometry}

\begin{document}
'''

for mooring in mooring_num:
    content += '\n' + tab_element_summary(mooring_df,df_data,mooring)
    content += '\n' + tab_clampon_sum(mooring_df,df_data,mooring_num,mooring)
    content += '\n' + tab_assembly_sum(mooring_df,df_data,mooring_num,mooring,bool_small=True)

content += '\n' + tab_total_instrument(df_merge)
content += '\n' + tab_summary_instrument(df_merge)
content += '\n' + tab_total_section_sum(df_merge,mooring_num)
content += '\n' + tab_per_sec(df_merge,mooring_num)
content += r'\end{document}'



print(content)

\documentclass{article}
\usepackage{graphicx} % Required for inserting images
\usepackage{tabularx}
\usepackage{adjustbox}
\usepackage[margin=.5in]{geometry}

\begin{document}

\begin{table}[]
\caption{W310 : Element summary}
\label{W310 : Element summary}
\centering
\begin{adjustbox}{max width=\textwidth,max height=0.8\textheight}
\centering
\begin{tabular}{llllllll}
\hline
Element & Qty & Length[m] & Buoyancy[kg] & Weight[kg] & SWL[T] & Comments \\ \hline
3M-WagonW & 1 &  & -900.0 &  &  &  \\
Ribuck Dual Kit for Sonardyne ORT & 1 &  & -35.0 &  &  &  \\
3/8 chain SL & 1 & 2.0 & -2.3 &  &  &  \\
100 cm Signature Frame & 1 &  & -7.5 &  &  &  \\
75kHz+40in+Ti300kHz & 1 &  & 85.8 &  &  &  \\
14in Panther Plast & 1 &  & 17.6 &  &  &  \\
30in & 2 &  & 149.0 &  &  &  \\
Sercel+frame & 1 &  & -10.0 &  &  &  \\
Swivel & 1 &  & -4.0 &  &  &  \\
13mmBS-L-16mmDS & 2 &  & -0.8 &  &  &  \\
150mm Load Ring & 1 &  & -0.8 &  &  &  \\
16mmBS & 5 &  & -0.7 &  &  &  \\
16mmDS & 1 &  & -0.7 &  &  &  \\
16

/tmp/ipykernel_212/2881566939.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ele['SERIAL NUMBER'].loc[idx] = ''
/tmp/ipykernel_212/2881566939.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ele['NAME'] = [i.replace(' (UWA)','').replace(' (passive)','') for i in df_ele['NAME']]


In [ ]:
with open('table_overview.tex', 'w') as f:
    f.write(content)
